## 7. Orchestration de la Conversation (Définitions des Stratégies)

Cette section définit les **classes** des stratégies qui gouverneront le déroulement de la conversation entre agents. Les instances seront créées lors de l'exécution.

*   **`SimpleTerminationStrategy` (Classe)** : Stratégie pour arrêter la conversation.
    *   **Condition d'arrêt :** Soit la `final_conclusion` est présente dans l'état partagé, soit un nombre maximum de tours (`max_steps`) est atteint.
    *   Initialisée avec l'instance de l'état partagé.

*   **`DelegatingSelectionStrategy` (Classe)** : Stratégie pour choisir quel agent parle au prochain tour.
    *   **Logique de sélection :**
        1.  Vérifie si un agent a été explicitement désigné dans l'état (`_next_agent_designated`). Si oui, le sélectionne.
        2.  Sinon (pas de désignation ou agent désigné invalide), applique une logique de fallback : retourne au `ProjectManagerAgent` (agent par défaut) après qu'un autre agent (spécialiste, utilisateur, outil) a parlé.
    *   Initialisée avec la liste des agents et l'instance de l'état partagé.

*   **`AgentGroupChat` (Classe de SK)** : Non définie ici, mais sera utilisée dans la cellule d'exécution pour gérer la conversation en utilisant les agents et les stratégies instanciés.

### 🚦 Classe Stratégie : SimpleTerminationStrategy

In [ ]:
# %% CELLULE [7.1] - Définition Classe SimpleTerminationStrategy
# (Remplace une partie de l'ancienne cellule 6b88c7ae)

from semantic_kernel.agents import Agent # Pour type hint Agent
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from typing import List, TYPE_CHECKING
import logging

# Type hinting
if TYPE_CHECKING:
    pass # Supposer RhetoricalAnalysisState dans scope global pour notebook

# Logger
termination_logger = logging.getLogger("Orchestration.Termination")
if not termination_logger.handlers and not termination_logger.propagate:
    handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); termination_logger.addHandler(handler); termination_logger.setLevel(logging.INFO)


class SimpleTerminationStrategy(TerminationStrategy):
    """Stratégie d'arrêt simple basée sur la conclusion ou le nombre max de tours."""
    _state: 'RhetoricalAnalysisState'
    _max_steps: int
    _step_count: int
    _instance_id: int

    def __init__(self, state: 'RhetoricalAnalysisState', max_steps: int = 15):
        """Initialise avec l'état partagé et le nombre max de tours."""
        super().__init__()
        if not hasattr(state, 'final_conclusion'):
             raise TypeError("Objet 'state' invalide pour SimpleTerminationStrategy.")
        self._state = state
        self._max_steps = max(1, max_steps)
        self._step_count = 0
        self._instance_id = id(self)
        self._logger = termination_logger
        self._logger.info(f"SimpleTerminationStrategy instance {self._instance_id} créée (max_steps={self._max_steps}, state_id={id(self._state)}).")

    async def should_terminate(self, agent: Agent, history: List[ChatMessageContent]) -> bool:
        """Vérifie si la conversation doit se terminer."""
        self._step_count += 1
        step_info = f"Tour {self._step_count}/{self._max_steps}"
        terminate = False
        reason = ""
        try:
            if self._state.final_conclusion is not None:
                terminate = True
                reason = "Conclusion finale trouvée dans l'état."
        except Exception as e_state_access:
             self._logger.error(f"[{self._instance_id}] Erreur accès état pour conclusion: {e_state_access}")
             terminate = False
        if not terminate and self._step_count > self._max_steps:
            terminate = True
            reason = f"Nombre max étapes ({self._max_steps}) atteint."
        if terminate:
            self._logger.info(f"[{self._instance_id}] Terminaison OUI. {step_info}. Raison: {reason}")
            return True
        else:
            self._logger.debug(f"[{self._instance_id}] Terminaison NON. {step_info}.")
            return False

    async def reset(self) -> None:
        """Réinitialise le compteur de tours."""
        self._logger.info(f"[{self._instance_id}] Reset SimpleTerminationStrategy (compteur {self._step_count} -> 0).")
        self._step_count = 0
        try:
            if self._state.final_conclusion is not None:
                 self._logger.warning(f"[{self._instance_id}] Reset strat, mais conclusion toujours présente dans état!")
        except Exception as e:
             self._logger.warning(f"[{self._instance_id}] Erreur accès état pendant reset: {e}")

logging.info("Classe SimpleTerminationStrategy définie.")


### 🚦 Classe Stratégie : DelegatingSelectionStrategy

In [ ]:
# %% CELLULE MODIFIÉE (ID c7a7ced5) - Définition Classe DelegatingSelectionStrategy (V2 - Fix Pydantic Init)

from semantic_kernel.agents import Agent
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from typing import List, Dict, TYPE_CHECKING
import logging
# *** CORRECTION: Importer PrivateAttr ***
from pydantic import PrivateAttr

# Type hinting
if TYPE_CHECKING:
    # Importez ou définissez RhetoricalAnalysisState ici si nécessaire pour TYPE_CHECKING
    # from path.to.state import RhetoricalAnalysisState # Exemple
    pass # Supposer RhetoricalAnalysisState dans scope global pour le notebook

# Logger
selection_logger = logging.getLogger("Orchestration.Selection")
if not selection_logger.handlers and not selection_logger.propagate:
    handler = logging.StreamHandler(); formatter = logging.Formatter('%(asctime)s [%(levelname)s] [%(name)s] %(message)s', datefmt='%H:%M:%S'); handler.setFormatter(formatter); selection_logger.addHandler(handler); selection_logger.setLevel(logging.INFO)


class DelegatingSelectionStrategy(SelectionStrategy):
    """Stratégie de sélection qui priorise la désignation explicite via l'état."""
    # *** CORRECTION: Utiliser PrivateAttr pour les champs gérés par __init__ ***
    _agents_map: Dict[str, Agent] = PrivateAttr()
    _default_agent_name: str = PrivateAttr(default="ProjectManagerAgent") # On peut mettre le défaut ici
    _analysis_state: 'RhetoricalAnalysisState' = PrivateAttr()
    # Ces deux-là ne semblent pas faire partie du modèle Pydantic de base, on peut les laisser
    # S'ils ne sont pas définis par Pydantic, ils doivent être explicitement typés
    _instance_id: int
    _logger: logging.Logger

    def __init__(self, agents: List[Agent], state: 'RhetoricalAnalysisState', default_agent_name: str = "ProjectManagerAgent"):
        """Initialise avec agents, état, et nom agent par défaut."""
        # L'appel super() doit rester ici
        super().__init__()
        if not isinstance(agents, list) or not all(isinstance(a, Agent) for a in agents):
            raise TypeError("'agents' doit être une liste d'instances Agent.")
        # S'assurer que la classe State est définie et que l'objet state a la bonne méthode
        if 'RhetoricalAnalysisState' not in globals() or not isinstance(state, RhetoricalAnalysisState) or not hasattr(state, 'consume_next_agent_designation'):
             raise TypeError("Objet 'state' invalide ou classe RhetoricalAnalysisState non définie pour DelegatingSelectionStrategy.")

        # *** CORRECTION: Assigner aux attributs privés ***
        self._agents_map = {agent.name: agent for agent in agents}
        self._analysis_state = state
        self._default_agent_name = default_agent_name # Le paramètre a priorité sur le défaut de PrivateAttr

        # Le reste de l'initialisation utilise maintenant les attributs privés
        self._instance_id = id(self)
        self._logger = selection_logger

        if self._default_agent_name not in self._agents_map:
            if not self._agents_map: raise ValueError("Liste d'agents vide.")
            first_agent_name = list(self._agents_map.keys())[0]
            self._logger.warning(f"[{self._instance_id}] Agent défaut '{self._default_agent_name}' non trouvé. Fallback -> '{first_agent_name}'.")
            self._default_agent_name = first_agent_name # Mettre à jour l'attribut privé

        self._logger.info(f"DelegatingSelectionStrategy instance {self._instance_id} créée (agents: {list(self._agents_map.keys())}, default: '{self._default_agent_name}', state_id={id(self._analysis_state)}).")

    async def next(self, agents: List[Agent], history: List[ChatMessageContent]) -> Agent:
        """Sélectionne le prochain agent à parler."""
        self._logger.debug(f"[{self._instance_id}] Appel next()...")
        # *** CORRECTION: Utiliser les attributs privés pour la logique ***
        default_agent_instance = self._agents_map.get(self._default_agent_name)
        if not default_agent_instance:
            self._logger.error(f"[{self._instance_id}] ERREUR: Agent défaut '{self._default_agent_name}' introuvable! Retourne premier agent.")
            available_agents = list(self._agents_map.values()) # Utilise _agents_map
            if not available_agents: raise RuntimeError("Aucun agent disponible.")
            return available_agents[0]

        try:
            # Utilise l'attribut privé _analysis_state
            designated_agent_name = self._analysis_state.consume_next_agent_designation()
        except Exception as e_state_access:
            self._logger.error(f"[{self._instance_id}] Erreur accès état pour désignation: {e_state_access}. Retour PM.")
            return default_agent_instance

        if designated_agent_name:
            self._logger.info(f"[{self._instance_id}] Désignation explicite: '{designated_agent_name}'.")
            # Utilise _agents_map
            designated_agent = self._agents_map.get(designated_agent_name)
            if designated_agent:
                self._logger.info(f" -> Sélection agent désigné: {designated_agent.name}")
                return designated_agent
            else:
                self._logger.error(f"[{self._instance_id}] Agent désigné '{designated_agent_name}' INTROUVABLE! Retour PM.")
                return default_agent_instance

        self._logger.debug(f"[{self._instance_id}] Pas de désignation. Fallback.")
        if not history:
            # Utilise _default_agent_name
            self._logger.info(f" -> Sélection (fallback): Premier tour -> Agent défaut ({self._default_agent_name}).")
            return default_agent_instance

        last_message = history[-1]
        last_author_name = getattr(last_message, 'name', getattr(last_message, 'author_name', None))
        last_role = getattr(last_message, 'role', AuthorRole.SYSTEM)
        self._logger.debug(f"   Dernier message: Role={last_role.name}, Author='{last_author_name}'")

        agent_to_select = default_agent_instance # Par défaut, on retourne au PM
        # Utilise _default_agent_name
        if last_role == AuthorRole.ASSISTANT and last_author_name != self._default_agent_name:
            self._logger.info(f" -> Sélection (fallback): Agent '{last_author_name}' a parlé -> Retour PM.")
        elif last_role == AuthorRole.USER:
             # Utilise _default_agent_name
            self._logger.info(f" -> Sélection (fallback): User a parlé -> Agent défaut ({self._default_agent_name}).")
        elif last_role == AuthorRole.TOOL:
             # Utilise _default_agent_name
             self._logger.info(f" -> Sélection (fallback): Outil a parlé -> Agent défaut ({self._default_agent_name}).")
        # Si le PM a parlé sans désigner, on retourne au PM (implicite car agent_to_select = default_agent_instance)
        else: # Autres cas ou PM a parlé sans désigner
            # Utilise _default_agent_name
            self._logger.info(f" -> Sélection (fallback): Rôle '{last_role.name}', Author '{last_author_name}' -> Agent défaut ({self._default_agent_name}).")

        self._logger.info(f" -> Agent sélectionné (fallback): {agent_to_select.name}")
        return agent_to_select

    async def reset(self) -> None:
        """Réinitialise la stratégie."""
        self._logger.info(f"[{self._instance_id}] Reset DelegatingSelectionStrategy.")
        try:
            # Utilise _analysis_state
            consumed = self._analysis_state.consume_next_agent_designation()
            if consumed: self._logger.debug(f"   Ancienne désignation '{consumed}' effacée.")
        except Exception as e:
            self._logger.warning(f"   Erreur accès état pendant reset sélection: {e}")

logging.info("Classe DelegatingSelectionStrategy (V2) définie.")

## 8. Exécution de la Conversation Collaborative (▶️ Lancement)

Cette cellule contient la fonction asynchrone `run_analysis_conversation` qui orchestre l'ensemble du processus :

1.  **Isolation:** Crée des instances *locales* et *neuves* pour chaque exécution (état, StateManager, Kernel, Agents, Stratégies, GroupChat).
2.  **Configuration Agents:** Crée les instances `ChatCompletionAgent` en utilisant le paramètre `service` (au lieu de `service_id`) et applique les `prompt_execution_settings` pour l'appel de fonction automatique.
3.  **Initialisation:** Prépare l'historique de conversation.
4.  **Lancement:** Démarre la boucle `local_group_chat.invoke()`.
5.  **Suivi:** Affiche chaque tour, indiquant l'agent (`message.name`) et le contenu. Logue les appels outils.
6.  **Terminaison:** Gérée par la `SimpleTerminationStrategy`.
7.  **Résultat:** Affiche l'historique et l'état final.

In [ ]:
# %% CELLULE MODIFIEE (ID c3702fe6) - Execution de la Conversation (V10.7 - Callback logging PL)

import time
import traceback
import asyncio
# import nest_asyncio # nest_asyncio est applique dans le notebook principal maintenant
from semantic_kernel.contents import ChatMessageContent, AuthorRole
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent, Agent
from semantic_kernel.exceptions import AgentChatException
import semantic_kernel as sk
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments
import logging
import json
import random
import jpype

# Logger principal
logger = logging.getLogger("Orchestration.Run")
if not logger.handlers and not logger.propagate: logger.addHandler(logging.StreamHandler()); logger.setLevel(logging.INFO)

# --- Verifications Prealables ---
required_globals = [
    'RhetoricalAnalysisState', 'StateManagerPlugin', 'ProjectManagerPlugin',
    'InformalAnalysisPlugin', 'PropositionalLogicPlugin',
    'SimpleTerminationStrategy', 'DelegatingSelectionStrategy',
    'global_ai_service_instance',
    'setup_pm_kernel', 'setup_informal_kernel', 'setup_pl_kernel',
    'PM_INSTRUCTIONS', 'INFORMAL_AGENT_INSTRUCTIONS', 'PL_AGENT_INSTRUCTIONS'
]
missing = [name for name in required_globals if name not in globals()]
if missing: raise NameError(f"Definitions manquantes: {', '.join(missing)}. Executez cellules precedentes.")
if not global_ai_service_instance: raise ValueError("Instance 'global_ai_service_instance' invalide.")
logger.info("Verifications prealables OK.")

# --- Fonction Principale d'Execution ---
async def run_analysis_conversation(texte_a_analyser: str):
    run_start_time = time.time()
    run_id = random.randint(1000, 9999)
    print("\n=====================================================")
    print(f"== Debut de l'Analyse Collaborative (Run_{run_id}) ==")
    print("=====================================================")
    run_logger = logging.getLogger(f"Orchestration.Run.{run_id}")
    run_logger.info("--- Debut Nouveau Run ---")

    local_state: Optional['RhetoricalAnalysisState'] = None
    local_kernel: Optional[sk.Kernel] = None
    local_group_chat: Optional[AgentGroupChat] = None
    local_state_manager_plugin: Optional[StateManagerPlugin] = None
    local_pl_plugin_instance: Optional[PropositionalLogicPlugin] = None  # Pour acces au query_log
    agent_list_local: List[Agent] = []

    try:
        # 1. Creer instance etat locale
        run_logger.info("1. Creation instance etat locale...")
        local_state = RhetoricalAnalysisState(initial_text=texte_a_analyser)
        run_logger.info(f"   Instance etat locale creee (id: {id(local_state)}) avec texte (longueur: {len(texte_a_analyser)}).")

        # 2. Creer instance StateManagerPlugin locale
        run_logger.info("2. Creation instance StateManagerPlugin locale...")
        local_state_manager_plugin = StateManagerPlugin(local_state)
        run_logger.info(f"   Instance StateManagerPlugin locale creee (id: {id(local_state_manager_plugin)}).")

        # 3. Creer Kernel local
        run_logger.info("3. Creation Kernel local...")
        local_kernel = sk.Kernel()
        local_kernel.add_service(global_ai_service_instance)
        run_logger.info(f"   Service LLM '{global_ai_service_instance.service_id}' ajoute.")
        local_kernel.add_plugin(local_state_manager_plugin, plugin_name="StateManager")
        run_logger.info(f"   Plugin 'StateManager' (local) ajoute.")

        # 4. Configurer plugins agents sur Kernel local
        run_logger.info("4. Configuration plugins agents sur Kernel local...")
        setup_pm_kernel(local_kernel, global_ai_service_instance)
        setup_informal_kernel(local_kernel, global_ai_service_instance)
        # V10.7: Passer le state_manager_plugin pour activer le callback de logging automatique
        local_pl_plugin_instance = setup_pl_kernel(local_kernel, global_ai_service_instance, state_manager_plugin=local_state_manager_plugin)
        run_logger.info("   Plugins agents configures (PL avec callback logging).")
        run_logger.debug(f"   Plugins enregistres: {list(local_kernel.plugins.keys())}")

        # 5. Creer instances agents locales
        run_logger.info("5. Creation instances agents...")
        prompt_exec_settings = local_kernel.get_prompt_execution_settings_from_service_id(global_ai_service_instance.service_id)
        prompt_exec_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(auto_invoke_kernel_functions=True, max_auto_invoke_attempts=5)
        run_logger.info(f"   Settings LLM (auto function call): {prompt_exec_settings.function_choice_behavior}")

        local_pm_agent = ChatCompletionAgent(
            kernel=local_kernel, service=global_ai_service_instance, name="ProjectManagerAgent",
            instructions=PM_INSTRUCTIONS, arguments=KernelArguments(settings=prompt_exec_settings)
        )
        local_informal_agent = ChatCompletionAgent(
            kernel=local_kernel, service=global_ai_service_instance, name="InformalAnalysisAgent",
            instructions=INFORMAL_AGENT_INSTRUCTIONS, arguments=KernelArguments(settings=prompt_exec_settings)
        )
        local_pl_agent = ChatCompletionAgent(
            kernel=local_kernel, service=global_ai_service_instance, name="PropositionalLogicAgent",
            instructions=PL_AGENT_INSTRUCTIONS, arguments=KernelArguments(settings=prompt_exec_settings)
        )
        agent_list_local = [local_pm_agent, local_informal_agent, local_pl_agent]
        run_logger.info(f"   Instances agents creees: {[agent.name for agent in agent_list_local]}.")

        # 6. Creer instances strategies locales
        run_logger.info("6. Creation instances strategies locales...")
        local_termination_strategy = SimpleTerminationStrategy(local_state, max_steps=15)
        local_selection_strategy = DelegatingSelectionStrategy(agents=agent_list_local, state=local_state)
        run_logger.info(f"   Instances strategies creees (Terminaison id: {id(local_termination_strategy)}, Selection id: {id(local_selection_strategy)}).")

        # 7. Creer instance AgentGroupChat locale
        run_logger.info("7. Creation instance AgentGroupChat locale...")
        local_group_chat = AgentGroupChat(
            agents=agent_list_local,
            selection_strategy=local_selection_strategy,
            termination_strategy=local_termination_strategy
        )
        run_logger.info(f"   Instance AgentGroupChat locale creee (id: {id(local_group_chat)}).")

        # 8. Initialiser historique et lancer invoke
        run_logger.info("8. Initialisation historique et lancement invoke...")
        initial_prompt = f"Bonjour a tous. Le texte a analyser est :\n'''\n{texte_a_analyser}\n'''\nProjectManagerAgent, merci de definir les premieres taches d'analyse en suivant la sequence logique."

        print(f"\n--- Tour 0 (Utilisateur) --- \n{initial_prompt}\n")
        run_logger.info(f"Message initial (Utilisateur): {initial_prompt}")

        # Ajouter le message initial a l'historique INTERNE du chat
        if hasattr(local_group_chat, 'history') and hasattr(local_group_chat.history, 'add_user_message'):
            local_group_chat.history.add_user_message(initial_prompt)
            run_logger.info("   Message initial ajoute a l'historique interne de AgentGroupChat.")
        else:
            run_logger.warning("   Impossible d'ajouter le message initial a l'historique interne de AgentGroupChat (attribut manquant?).")

        invoke_start_time = time.time()
        run_logger.info(">>> Debut boucle invocation AgentGroupChat <<<")
        turn = 0

        # --- Boucle Invoke ---
        async for message in local_group_chat.invoke():
            turn += 1
            if not message:
                run_logger.warning(f"Tour {turn}: Invoke a retourne un message vide. Arret.")
                break

            author_display_name = message.name or getattr(message, 'author_name', f"Role:{message.role.name}")
            role_display_name = message.role.name

            print(f"\n--- Tour {turn} ({author_display_name} / {role_display_name}) ---")
            run_logger.info(f"----- Debut Tour {turn} - Agent/Author: '{author_display_name}', Role: {role_display_name} -----")

            content_str = str(message.content) if message.content else ""
            content_display = content_str[:500] + "..." if len(content_str) > 500 else content_str
            print(f"  Content: {content_display}")
            run_logger.debug(f"  Msg Content T{turn} (Full): {content_str}")

            tool_calls = getattr(message, 'tool_calls', []) or []
            if tool_calls:
                print("   Tool Calls:")
                run_logger.info("   Tool Calls:")
                for tc in tool_calls:
                    plugin_name, func_name = 'N/A', 'N/A'
                    function_name_attr = getattr(getattr(tc, 'function', None), 'name', None)
                    if function_name_attr and isinstance(function_name_attr, str) and '-' in function_name_attr:
                        parts = function_name_attr.split('-', 1)
                        if len(parts) == 2: plugin_name, func_name = parts
                    args_dict = getattr(getattr(tc, 'function', None), 'arguments', {}) or {}
                    args_str = json.dumps(args_dict) if args_dict else "{}"
                    args_display = args_str[:200] + "..." if len(args_str) > 200 else args_str
                    log_msg_tc = f"     - ID: {getattr(tc, 'id', 'N/A')}, Func: {plugin_name}-{func_name}, Args: {args_display}"
                    print(log_msg_tc)
                    run_logger.info(log_msg_tc)
                    run_logger.debug(f"     - Tool Call Full Args: {args_str}")

            await asyncio.sleep(0.05)

        invoke_duration = time.time() - invoke_start_time
        run_logger.info(f"<<< Fin boucle invocation ({invoke_duration:.2f} sec) >>>")
        print("\n--- Conversation Terminee ---")

    except AgentChatException as chat_complete_error:
        if "Chat is already complete" in str(chat_complete_error):
            run_logger.warning(f"Chat deja termine: {chat_complete_error}")
            print("\n Chat deja marque comme termine.")
        else:
            run_logger.error(f"Erreur AgentChatException: {chat_complete_error}", exc_info=True)
            print(f"\n Erreur AgentChatException : {chat_complete_error}")
            traceback.print_exc()
    except Exception as e:
        run_logger.error(f"Erreur majeure execution conversation: {e}", exc_info=True)
        print(f"\n Erreur majeure : {e}")
        traceback.print_exc()
    finally:
        # --- Affichage Final ---
        run_end_time = time.time()
        total_duration = run_end_time - run_start_time
        run_logger.info(f"Fin analyse. Duree totale: {total_duration:.2f} sec.")

        print("\n--- Historique Detaille ---")
        final_history_messages = []
        if local_group_chat and hasattr(local_group_chat, 'history') and hasattr(local_group_chat.history, 'messages'):
            final_history_messages = local_group_chat.history.messages

        if final_history_messages:
            for msg in final_history_messages:
                author = msg.name or getattr(msg, 'author_name', f"Role:{msg.role.name}")
                role_name = msg.role.name
                content_display = str(msg.content)[:500] + "..." if len(str(msg.content)) > 500 else str(msg.content)
                print(f"[{author} ({role_name})]: {content_display}")
                tool_calls = getattr(msg, 'tool_calls', []) or []
                if tool_calls:
                    print("   Tool Calls:")
                    for tc in tool_calls:
                        plugin_name, func_name = 'N/A', 'N/A'
                        function_name_attr = getattr(getattr(tc, 'function', None), 'name', None)
                        if function_name_attr and isinstance(function_name_attr, str) and '-' in function_name_attr:
                            parts = function_name_attr.split('-', 1)
                            if len(parts) == 2: plugin_name, func_name = parts
                        args_dict = getattr(getattr(tc, 'function', None), 'arguments', {}) or {}
                        args_str = json.dumps(args_dict) if args_dict else "{}"
                        args_display = args_str[:100] + "..." if len(args_str) > 100 else args_str
                        print(f"     - {plugin_name}-{func_name}({args_display})")
        else:
            print("(Historique final vide ou inaccessible)")
        print("---------------------------\n")

        # V10.7: Afficher le log interne des requetes PL si disponible
        if local_pl_plugin_instance:
            pl_query_log = local_pl_plugin_instance.get_query_log()
            if pl_query_log:
                print("\n--- Log Requetes PL (interne plugin) ---")
                for entry in pl_query_log:
                    print(f"  [{entry.get('timestamp', 'N/A')}] {entry.get('query', 'N/A')} -> {entry.get('result', 'N/A')}")
                print("----------------------------------------\n")
                run_logger.info(f"Log requetes PL: {len(pl_query_log)} entrees")
            else:
                run_logger.info("Log requetes PL vide (aucune requete executee via execute_and_log_pl_query)")

        print("=========================================")
        print("== Fin de l'Analyse Collaborative ==")
        print(f"== Duree: {total_duration:.2f} secondes ==")
        print("=========================================")
        print("\n--- Etat Final de l'Analyse (Instance Locale) ---")
        if local_state:
            try: print(local_state.to_json(indent=2))
            except Exception as e_json: print(f"(Erreur serialisation etat final: {e_json})"); print(f"Repr: {repr(local_state)}")
        else: print("(Instance etat locale non disponible)")

        jvm_status = "(JVM active)" if ('jpype' in globals() and jpype.isJVMStarted()) else "(JVM non active)"
        print(f"\n{jvm_status}")
        run_logger.info(f"Etat final JVM: {jvm_status}")
        run_logger.info(f"--- Fin Run_{run_id} ---")

    # Retourner l'etat local pour le notebook Executor
    return local_state

# # L'appel reel est commente car il sera fait depuis le notebook principal
# nest_asyncio.apply() # Est deja applique dans le notebook principal
# asyncio.run(run_analysis_conversation("Texte test ici si execute seul"))

## 9. 🏁 Conclusion et Prochaines Étapes

L'état final affiché après l'exécution de la cellule précédente montre les résultats de l'analyse collaborative effectuée par les agents, après correction de l'erreur d'instanciation.

**Observations Attendues sur cette exécution :**
*   ✅ **Exécution:** Le notebook devrait maintenant s'exécuter sans l'erreur `TypeError` lors de la création des agents.
*   ✅ **Affichage Tours:** Vérifier si les tours affichent le nom correct de l'agent (e.g., `[ProjectManagerAgent (ASSISTANT)]`).
*   ✅ **Désignation Agent:** Vérifier l'absence d'erreurs "Agent ... introuvable" dans les logs `stderr`.
*   ✅ **Type Logique:** Vérifier l'absence d'erreurs "Type logique ... non supporté" lors de l'appel à `StateManager.add_belief_set`.
*   ➡️ **Séquence Analyse:** Observer si la séquence d'analyse est suivie de manière logique par le `ProjectManagerAgent`.
*   ➡️ **Conclusion Finale:** Noter si la conclusion est générée au bon moment et basée sur une analyse complète.
*   ⚠️ **JVM / Tweety:** Si la JVM n'était pas prête (cellule 2), les fonctions du `PropositionalLogicAgent` utilisant `execute_pl

**Prochaines étapes possibles :**
*   **Activer & Finaliser PL:** Implémenter réellement les appels JPype/Tweety dans `PropositionalLogicPlugin._internal_execute_query` et tester de bout en bout l'exécution des requêtes logiques (parsing, query, interprétation).
*   **Affiner Analyse Sophismes:** Améliorer les instructions de `InformalAnalysisAgent` pour une exploration plus fine de la taxonomie (gestion de la profondeur, choix des branches) ou l'attribution de sophismes spécifiques basée sur les détails récupérés (`get_fallacy_details`).
*   **Externaliser Prompts & Config:** Déplacer les prompts et configurations (ex: noms agents, constantes) hors du code Python vers des fichiers dédiés (YAML, JSON, .env) pour une meilleure maintenabilité. Utiliser `kernel.import_plugin_from_directory`.
*   **Gestion Erreurs Agents:** Renforcer la capacité des agents à gérer les erreurs retournées par les outils (`FUNC_ERROR:`) et à adapter leur plan (ex: demander une clarification, réessayer, passer à autre chose).
*   **Nouveaux Agents/Capacités:** Implémenter des agents pour d'autres logiques (FOL, Modale), d'autres tâches (résumé, extraction d'entités) ou d'autres outils (recherche web, base de données).
*   **État RDF/KG:** Explorer le passage à une structure d'état plus riche et sémantiquement structurée en utilisant RDF/KG (avec `rdflib` ou une base de graphe) pour représenter les arguments, relations, et métadonnées de manière plus formelle.
*   **Interface Utilisateur:** Créer une interface (ex: avec Gradio, Streamlit) pour faciliter l'interaction et la visualisation de l'analyse.